In [ ]:
#directions = [5,15,20,25,35]
#labels = ['extremeleft','left','middle','right','extremeright']

directions = [0,5,10,15,20,25,30,35]
N = len(directions)
#labels = ['baseline','5','10','15','middle','25','30','extremeright']
#labeldict = dict(zip(directions,labels))

n_each = 20 #15

In [ ]:
N_PARALLEL = 28 # for parallel processing, how many cores can we use

In [ ]:
%pylab inline
import random
import pandas as pd
import numpy as np
import time
import multiprocessing
import ctypes
import numpy as np
from IPython.display import display, clear_output
import seaborn as sns
import os

In [ ]:
def create_random_design():
    dirs = []
    lastdir = None
    for _ in range(n_each):
        acceptable = False
        while not acceptable:
            d = list(range(N)) #directions[:]
            random.shuffle(d)
            acceptable = d[0]!=lastdir
        lastdir=d[-1]
        dirs+=d
    return dirs

In [ ]:
dirs = create_random_design()
" ".join([str(directions[d]) for d in dirs ])

In [ ]:
len(dirs)

## What are the carry-over effects?

In [ ]:
def get_transitions(dirs):
    # Get the first-order transition matrix
    trans = [ (dirs[i],dirs[i+1]) for i in range(len(dirs)-1) ]
    #n = len(directions)
    transcount = np.zeros((N,N))
    for i in range(len(dirs)-1):
        transcount[dirs[i],dirs[i+1]]+=1
    return transcount

In [ ]:
transcount = get_transitions(dirs)
transcount

In [ ]:
imshow(transcount)

In [ ]:
sum(transcount,0)

In [ ]:
#sum(transcount,0)

In [ ]:
20./7

In [ ]:
3*6+1*2

## Batch-generate schedules and pick the best one
Generations of scholars will wonder why on Earth we do this, but hey...

In [ ]:
def badness(mat):
    # Tell us how bad this transition matrix is - how unequal are the transitional probability distributions?
    
    # Compute the average of each row
    mat = np.array(mat)
    n,_ = mat.shape
    # Restrict to the off-diagonal elements 
    rowvalues = [ vals[sel] for (vals,sel) in zip(mat,(1-np.eye(n)).astype(bool)) ]
    
    # Compute the deviation of each item from the average for that row
    dists = [ [ abs(r-np.mean(row)) for r in row ] for row in rowvalues ]
    
    return sum(np.array(dists).flatten())

In [ ]:
badness(transcount)

### What is the best badness we can expect?

In [ ]:
theo = 3*(ones((N,N))*(1-eye(N)))
for i in range(N):
    if i==3:
        theo[i,4]=2
        theo[i,2]=2
    else:
        theo[i,3]=2

In [ ]:
theo

In [ ]:
#sum(theo)

In [ ]:
imshow(theo)

In [ ]:
sum(theo,1)

In [ ]:
LOWEST = badness(theo)
LOWEST

In [ ]:
#N_DESIGNS = 1000000 # Neeraj wanted 100000000
N_DESIGNS = 10000

# Distribution of badnesses
shared_array_base = multiprocessing.Array(ctypes.c_double, N_DESIGNS)
shared_array = np.ctypeslib.as_array(shared_array_base.get_obj())
#shared_array = shared_array.reshape(10, 10)

# Best the design
shared_design_base = multiprocessing.Array(ctypes.c_int, n_each*len(directions))
shared_design = np.ctypeslib.as_array(shared_design_base.get_obj())

bestbad = multiprocessing.Value('d', 999999)


def generate_random(i,allbadness=shared_array,bestdesign=shared_design,bestb=bestbad):
    #print(i)
    design = create_random_design()
    trans = get_transitions(design)
    howbad = badness(trans)
    allbadness[i]=howbad
    if howbad<bestb.value:
        bestb.value=howbad
        for j,v in enumerate(design):
            shared_design[j]=v
    if i%100000==0:
        print("{} best {}".format(i,bestbad.value))
    return 

# Parallel processing
#def my_func(i, def_param=shared_array):
#    shared_array[i,:] = i

t0 = time.time()

pool = multiprocessing.Pool(processes=N_PARALLEL)
pool.map(generate_random, range(N_DESIGNS))

tsec = (time.time()-t0)
tdur = tsec/(60*60)

print("This took {} seconds i.e. {} hours for {} designs".format(tsec,tdur,N_DESIGNS))
targetdes = 100000000
print("At this rate {} will take {} hours.".format(targetdes,targetdes*tdur/N_DESIGNS))

#print(shared_array)
#bestbad_posthoc = min(shared_array)
#print(bestbad_posthoc,bestbad.value)

In [ ]:
sns.distplot(shared_array[:1000000])
axvline(x=bestbad.value,color='red')
axvline(x=LOWEST,color='green')
sns.despine(offset=5)
print(bestbad.value)


In [ ]:
dirs = list(shared_design)

In [ ]:
transcount = get_transitions(dirs)
transcount

In [ ]:
imshow(transcount)

In [ ]:
# Let's save the history
import pickle
PICKLE_F = 'history.pickle'
if os.path.exists(PICKLE_F):
    history = pickle.load(open(PICKLE_F,'rb'))
else:
    history = []
history.append({"source":"bruteforce","best_design":list(shared_design)})
pickle.dump(history,open(PICKLE_F,'wb'))

Find the best design from the history...

# Making sub-designs combining them into larger ones

So the idea here is to create smaller designs that hopefully will be pretty good and then combining them into larger designs, hoping that those are good too!

In [ ]:
SUBSIZE  = 10 # instead of N_EACH, we make smaller designs of SUBSIZE * directions which we try to optimize
N_SEARCH = 1000000 # create a whole lot of random small sub designs
#N_SEARCH = 1000
N_PICK   = 1000 # subset of the best of the N_SEARCH designs from which we start making our MASTER design

In [ ]:
assert n_each%SUBSIZE==0

In [ ]:
MINI_SIZE = SUBSIZE*len(directions) # how big a mini design is

## STEP 1: GENERATE N_PICK DESIGNS

If we want to save all generated designs, we need `N_SEARCH * SUBSIZE * directions` array. That's too much, and we need to go through them anyway afterwards.

In [ ]:
t0 = time.time()

# Best the design
ALLDESIGNSSIZE = N_PICK*MINI_SIZE
minidesigns_base = multiprocessing.Array(ctypes.c_int, ALLDESIGNSSIZE)
minidesigns = np.ctypeslib.as_array(minidesigns_base.get_obj())


def create_random_minidesign():
    dirs = []
    lastdir = None
    for _ in range(SUBSIZE):
        acceptable = False
        while not acceptable:
            d = list(range(N)) #directions[:]
            random.shuffle(d)
            acceptable = d[0]!=lastdir
        lastdir=d[-1]
        dirs+=d
    return dirs



def generate_random_static(i,minidesigns=minidesigns):
    current_bad = 99999
    current_design = None

    print("Iteration {}".format(i))
    
    for n in range(N_SEARCH):
        design = create_random_minidesign()
        trans = get_transitions(design)
        howbad = badness(trans)
        if howbad<current_bad:
            current_bad=howbad
            current_design = design[:]
    #print(current_design)
    
    # The result of this function is a "good" mini-design
    # Insert it into the current list of minidesigns
    x = i*MINI_SIZE
    for j,d in enumerate(current_design):
        minidesigns[x+j]=d
        
    return


pool = multiprocessing.Pool(processes=N_PARALLEL)
pool.map(generate_random_static, range(N_PICK))
#pool.map(generate_random_static, range(2))
            
tsec = (time.time()-t0)
tdur = tsec/(60*60)

print("This took {} seconds i.e. {} hours".format(tsec,tdur))

In [ ]:
minidesigns = minidesigns.reshape( (N_PICK,MINI_SIZE) )

In [ ]:
imshow(minidesigns)

In [ ]:
minidesigns = [ list(minidesigns[i,:]) for i in range(N_PICK) ]

In [ ]:
def design_to_badness(d):
    if sum(d)==0: return 99999
    transcount = get_transitions(d)
    return badness(transcount)

In [ ]:
minibad = [ design_to_badness(md) for md in minidesigns ]

sns.distplot(minibad)
#axvline(x=bigbestbadness,color='red')
#axvline(x=LOWEST,color='green')
sns.despine(offset=5)
print(bestbad.value)


In [ ]:
#minidesigns
if False:
    pickle.dump(minidesigns,open('minidesigns.pickle','wb'))

## STEP 2: COMBINE CHILDREN TO MAKE MASTER DESIGN

In [ ]:
matrix(minidesigns)
imshow(minidesigns)

In [ ]:
#minidesigns[0]'Iteration 532 badness 28.5714285714'

In [ ]:
def create_master_design():
    # randomly create master design from the children
    # but check that the edges align, i.e. that we don't create
    # transitions from a particular direction to itself
    
    des = []
    while len(des)<n_each*len(directions):
        
        # concatenate another design
        mini = random.choice(minidesigns)
        
        # can we actually append this?
        if len(des) and des[-1]==mini[0]:
            continue # we cannot append this!
        
        des += mini

    return des

In [ ]:
#minidesigns[1]

In [ ]:
m = create_master_design()

In [ ]:
len(m)

In [ ]:
#t = get_transitions(m)[3., 3., 4., 2., 2., 2., 0., 4.],

In [ ]:
#badness(t)

In [ ]:
N_MONTE_CARLO = 1000000

bestmaster = None
bestmasterbad = 99999

graphbest = zeros(N_MONTE_CARLO)

for i in range(N_MONTE_CARLO):
    
    master = create_master_design() # create a master design (concat)
    
    # determine badness of master
    howbad = design_to_badness(master)
    
    # if better than before, keep it as best
    if howbad<bestmasterbad:
        bestmaster = master[:]
        bestmasterbad = howbad

    if i%100==0:
        clear_output(wait=True)
        display('Iteration {} badness {}'.format(i,bestmasterbad))
    
    graphbest[i]= bestmasterbad

In [ ]:
plot(graphbest)

In [ ]:
" ".join([ str(s) for s in bestmaster])

In [ ]:
trans = get_transitions(bestmaster)
trans

In [ ]:
imshow(trans)

In [ ]:
sns.distplot(shared_array[:1000000])
#axvline(x=bigbestbadness,color='red')
axvline(x=bestmasterbad,color='purple')
axvline(x=LOWEST,color='green')
sns.despine(offset=5)
#print(bestbad.value)

In [ ]:
# Let's save the history
import pickle
PICKLE_F = 'history.pickle'
if os.path.exists(PICKLE_F):
    history = pickle.load(open(PICKLE_F,'rb'))
else:
    history = []
history.append({"minidesigns":minidesigns,"SUBSIZE":SUBSIZE,"N_SEARCH":N_SEARCH,"N_PICK":N_PICK,"source":"recombine_smaller","best_design":list(bestmaster)})
pickle.dump(history,open(PICKLE_F,'wb'))

# The whole history & Output
What is the best thing we have found using any method?

In [ ]:
print("{} candidates in history".format(len(history)))

In [ ]:
#design_to_badness(h['best_design'])

In [ ]:
for h in history:
    h['badness']= design_to_badness(h['best_design'])
history = sorted(history, 
                 key = lambda x: x['badness'])

In [ ]:
#history
bigbest = history[0]['best_design']
bigbestbadness = history[0]['badness']
bigbestbadness

In [ ]:
" ".join([ str(directions[b]) for b in bigbest ])

In [ ]:
sns.distplot(shared_array[:1000000])
axvline(x=bigbestbadness,color='red')
axvline(x=LOWEST,color='green')
text(bigbestbadness,0,'%.2f'%bigbestbadness,color='red')
sns.despine(offset=5)
print(bestbad.value)

## Output

In [ ]:
chosen = [ directions[d] for d in bigbest ]
' '.join([str(s) for s in chosen])

In [ ]:
tab = pd.DataFrame({'direction':chosen})
tab['type']=[ 'direction{}'.format(d) for d in tab['direction']]
tab['trial']=np.arange(len(chosen))+1
#tab.to_csv('recognitionyesno_theOne.csv')
outcsv = 'recognitionyesno_bestoverall_%.2f.csv'%bigbestbadness
tab.to_csv(outcsv)
print(outcsv)

In [ ]:
tab.head()

## Double-check
I know, not strictly necessary, but it sounds like a good idea to double-check what we have written out.

In [ ]:
drs = pd.read_csv('recognitionyesno_efficient23.csv')['direction']
thedirs = list(set(drs))
dr = [ thedirs.index(d) for d in drs ]

In [ ]:
#drs
trans = get_transitions(dr)
imshow(trans)

In [ ]:
badness(trans)

# Doing this the smart way (maybe)

In [ ]:
if True:
    #Highest number in square
    order_of_sq = int(input("Enter order of sq: "))

    #Number you want to start the square with
    top_left = int(input("Enter top left number: "))

    #Sets a placeholder for a variable called top_left_init
    top_left_init=0

    #Sets the initial value of top_left to a new variable because the code will change the value of top left later on 
    top_left_init += top_left

    #Initialize the value of count
    count = 0

    #Add 1 to the highest value in latin square to account for the range function (the ending number is always one less than the number you enter into the range function)
    for values in range (1,order_of_sq+1):

        #Prevents the program from adding too many characters to the line
        while count != order_of_sq:

            #Prints numbers with spaces after them in a horizontal manner
            print(top_left,sep=' ',end=' ')

            #Adds 1 to the top_left
            top_left += 1

            #Count is used to keep track of how many characters are in your line
            count+=1

            #Restarts the numbers in your line when you reach the highest number
            if top_left == order_of_sq+1:
                top_left = 1

        #Creates a new row
        print()
        count = 0

        #Calls the initial value of top_left and adds 1 to it before beginning the next row
        top_left_init += 1

        #Resets top_left_init to 1 if it reaches the highest number in the square
        if top_left_init == order_of_sq + 1:
            top_left_init = 1
            top_left = top_left_init
        else:
            top_left = top_left_init